In [303]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr

In [304]:
Predicted = pd.read_csv("./Seurat/Outputs/output_Sim8.csv")
Target=pd.read_csv("./Data/Sim8/target.csv")

In [305]:
Predicted

,Unnamed: 0,prediction.score.Endothelial.Cell,prediction.score.Thick.Ascending.Limb,prediction.score.Distal.Convoluted.Tubule,prediction.score.Proximal.Tubule,prediction.score.Connecting.Tubule,prediction.score.Principal.Cells,prediction.score.Intercalated.Cells.Type.A,prediction.score.Intercalated.Cells.Type.B,prediction.score.Mesangial.Cells,prediction.score.Leukocyte,prediction.score.max
0,0,0.000000,0.084338,0.007880,0.353459,0.016337,0.281324,0.141052,0.072762,0.0,0.042849,0.353459
1,1,0.056483,0.524937,0.000000,0.415961,0.000000,0.000000,0.000000,0.002618,0.0,0.000000,0.524937
2,2,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000
3,3,0.007069,0.038677,0.000000,0.940317,0.005713,0.007044,0.000000,0.000000,0.0,0.001181,0.940317
4,4,0.362362,0.000000,0.001541,0.069885,0.000000,0.503152,0.006900,0.050614,0.0,0.005545,0.503152
...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,0.041857,0.000000,0.000000,0.958143,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.958143
996,996,0.000000,0.003393,0.037068,0.014554,0.944985,0.000000,0.000000,0.000000,0.0,0.000000,0.944985
997,997,0.000000,0.032045,0.000000,0.047559,0.018204,0.879799,0.000000,0.022393,0.0,0.000000,0.879799
998,998,0.921192,0.000000,0.000000,0.071625,0.000000,0.000000,0.000000,0.000000,0.0,0.007183,0.921192


In [306]:
Target

,Unnamed: 0,Proximal Tubule,Thick Ascending Limb,Distal Convoluted Tubule,Connecting Tubule,Principal Cells,Intercalated Cells Type A,Intercalated Cells Type B,Endothelial Cell,Mesangial Cells,Leukocyte
0,0,5.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0
1,1,5.0,4.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
2,2,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
995,995,9.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
996,996,2.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
997,997,3.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0
998,998,9.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


In [307]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
Predicted.drop(columns="prediction.score.max", inplace=True)


In [308]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [309]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [310]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [311]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [312]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [313]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [314]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [315]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [316]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

In [317]:
np.mean(np.nan_to_num(d))

0.6509684372071496

In [318]:
d

array([0.62683282, 0.67709571, 0.76933754, 0.55148372, 0.5451455 ,
       0.4435543 , 0.81144884, 0.62713281, 0.65800868, 0.79964446])

## Jenson Shannon Divergence

In [319]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.39262902758142587

## F1 Score, Precision, Recall

In [320]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [321]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [322]:
d

,Precision,Recall,F1-score
0.100,0.831776,0.540937,0.655546
0.200,0.855062,0.529067,0.653675
0.300,0.753281,0.558366,0.641341
0.010,0.587589,0.734134,0.652738
0.001,0.533771,0.780789,0.634072
